<a href="https://colab.research.google.com/github/YuliyaStarostina/ETL/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22ETL_HW_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from math import asin
from typing import Union
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power
from scipy.stats import ttest_1samp
import numpy as np
from scipy.stats import stats
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')
from statsmodels.stats.power import TTestIndPower
from __future__ import division
from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power
import operator
import numpy as np
import pandas as pn


# Урок 2. Введение в подготовку данных для аналитиков. Таблицы фактов и таблицы измерений

1. Скачайте датасет fifаs2.сsv. Проанализируйте его и определите, какие данные являются неполными. Удалите
ненужные колонки и недостающие значения.

2. Найдите в датафрейме полные дубликаты и удалите их. Значения могут быть одинаковыми, но написаны по-разному. Например, может отличаться размер регистра (заглавные и строчные буквы). Особое внимание уделить колонке с названиями команд.

3. Напишите функцию, которая добавит колонку с разбиением возраста по группам: до 20, от 20 до 30, от 30 до
36 и старше 36. Посчитайте количество футболистов в каждой категории.

# 1. Скачайте датасет fifаs2.сsv. Проанализируйте его и определите, какие данные являются неполными. Удалите ненужные колонки и недостающие значения.

In [71]:
df = pd.read_csv('/content/fifa_s2 (1) (1).csv')
df.head(5)



,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Skill Moves,Position,Joined,Contract Valid Until,Height,Weight,Release Clause
0,1179,G. Buffon,40,Italy,88,88,Paris Saint-Germain,4000.0,77,Right,4.0,1.0,GK,2018,01/01/2019,6.333333,203.0,7400.0
1,5479,Casillas,37,Spain,82,82,FC Porto,1500.0,10,Left,4.0,1.0,NaN,2015,01/01/2019,6.083333,185.0,3000.0
2,8205,Braga,35,Portugal,70,70,CD Aves,750.0,5,Right,1.0,3.0,LCM,2017,01/01/2019,5.916667,165.0,1600.0
3,9014,A. Robben,34,Netherlands,84,84,FC Bayern München,15500.0,110,Left,4.0,4.0,RM,2009,01/01/2019,5.916667,176.0,25600.0
4,19521,J. Ibehre,35,England,61,61,Cambridge United,120.0,2,Right,1.0,2.0,ST,2017,01/01/2019,6.333333,203.0,210.0


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2399 entries, 0 to 2398
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        2399 non-null   int64  
 1   Name                      2399 non-null   object 
 2   Age                       2399 non-null   int64  
 3   Nationality               2392 non-null   object 
 4   Overall                   2399 non-null   int64  
 5   Potential                 2399 non-null   int64  
 6   Club                      2158 non-null   object 
 7   Value                     2142 non-null   float64
 8   Wage                      2399 non-null   int64  
 9   Preferred Foot            2399 non-null   object 
 10  International Reputation  2389 non-null   float64
 11  Skill Moves               2393 non-null   float64
 12  Position                  2378 non-null   object 
 13  Joined                    2399 non-null   int64  
 14  Contract

### Неполными являются данные в столбцах: Club, Value, Contract Valid Until. Незначительный процент пропусков содержит Position

In [40]:
#автоматический подбор типа данных
sample = pd.read_csv('/content/fifa_s2 (1) (1).csv', usecols=['Name', 'Nationality', 'Club', 'Value','Preferred Foot', 'Position', 'Contract Valid Until'])
sample.convert_dtypes().dtypes


Name                    string
Nationality             string
Club                    string
Value                    Int64
Preferred Foot          string
Position                string
Contract Valid Until    string
dtype: object

In [41]:
# процент пропущенных значений
# для этого разделим сумму пропусков в каждом столбце на количество наблюдений,
# округлим результат и умножим его на 100
(df.isna().sum() / len(df)).round(4) * 100

ID                           0.00
Name                         0.00
Age                          0.00
Nationality                  0.29
Overall                      0.00
Potential                    0.00
Club                        10.05
Value                       10.71
Wage                         0.00
Preferred Foot               0.00
International Reputation     0.42
Skill Moves                  0.25
Position                     0.88
Joined                       0.00
Contract Valid Until        11.38
Height                       0.00
Weight                       0.00
Release Clause               0.13
dtype: float64

In [42]:
import seaborn as sns # установление зависимости между данными
df_corr = df.corr().head(50)
cm = sns.light_palette('green', as_cmap=True)
df_corr.style.text_gradient(cmap=cm)




,ID,Age,Overall,Potential,Value,Wage,International Reputation,Skill Moves,Joined,Height,Weight,Release Clause
ID,1.000000,-0.752822,-0.454069,-0.057004,-0.207231,-0.288699,-0.429858,-0.034038,0.255051,-0.120178,-0.241487,-0.192642
Age,-0.752822,1.000000,0.409760,-0.194086,0.114994,0.170866,0.301181,-0.020392,-0.229523,0.086351,0.231786,0.090264
Overall,-0.454069,0.409760,1.000000,0.749045,0.723444,0.690481,0.697814,0.419883,-0.257649,0.094880,0.186376,0.686498
Potential,-0.057004,-0.194086,0.749045,1.000000,0.664133,0.609459,0.555550,0.399757,-0.125230,0.058171,0.051954,0.637561
Value,-0.207231,0.114994,0.723444,0.664133,1.000000,0.899048,0.765076,0.369730,-0.215261,0.017389,0.078386,0.976644
Wage,-0.288699,0.170866,0.690481,0.609459,0.899048,1.000000,0.789622,0.323130,-0.266748,0.031702,0.091979,0.883674
International Reputation,-0.429858,0.301181,0.697814,0.555550,0.765076,0.789622,1.000000,0.283784,-0.276681,0.060880,0.129962,0.734948
Skill Moves,-0.034038,-0.020392,0.419883,0.399757,0.369730,0.323130,0.283784,1.000000,-0.028300,-0.396758,-0.349713,0.343468
Joined,0.255051,-0.229523,-0.257649,-0.125230,-0.215261,-0.266748,-0.276681,-0.028300,1.000000,-0.008834,-0.053900,-0.206248
Height,-0.120178,0.086351,0.094880,0.058171,0.017389,0.031702,0.060880,-0.396758,-0.008834,1.000000,0.778196,0.014444


### Самая высокая корелляция (0.8+) между показателями Release Clause и Value, Release Clause и Wage, Wage и Value

In [72]:
# Contract Valid Until имеет большое число пропусков, при этом не оказывает влияния на остальные показатели.
# Можем его удалить
df.drop(columns = ['Contract Valid Until'], axis=1, inplace = True)
df.to_csv('upd_dataset_new.csv', index = False)
print('колонка успешно удалена')

колонка успешно удалена


In [73]:
df =pd.read_csv('/content/upd_dataset_new.csv')
df.head(5)

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Skill Moves,Position,Joined,Height,Weight,Release Clause
0,1179,G. Buffon,40,Italy,88,88,Paris Saint-Germain,4000.0,77,Right,4.0,1.0,GK,2018,6.333333,203.0,7400.0
1,5479,Casillas,37,Spain,82,82,FC Porto,1500.0,10,Left,4.0,1.0,NaN,2015,6.083333,185.0,3000.0
2,8205,Braga,35,Portugal,70,70,CD Aves,750.0,5,Right,1.0,3.0,LCM,2017,5.916667,165.0,1600.0
3,9014,A. Robben,34,Netherlands,84,84,FC Bayern München,15500.0,110,Left,4.0,4.0,RM,2009,5.916667,176.0,25600.0
4,19521,J. Ibehre,35,England,61,61,Cambridge United,120.0,2,Right,1.0,2.0,ST,2017,6.333333,203.0,210.0


In [74]:
df.head(5)

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Skill Moves,Position,Joined,Height,Weight,Release Clause
0,1179,G. Buffon,40,Italy,88,88,Paris Saint-Germain,4000.0,77,Right,4.0,1.0,GK,2018,6.333333,203.0,7400.0
1,5479,Casillas,37,Spain,82,82,FC Porto,1500.0,10,Left,4.0,1.0,NaN,2015,6.083333,185.0,3000.0
2,8205,Braga,35,Portugal,70,70,CD Aves,750.0,5,Right,1.0,3.0,LCM,2017,5.916667,165.0,1600.0
3,9014,A. Robben,34,Netherlands,84,84,FC Bayern München,15500.0,110,Left,4.0,4.0,RM,2009,5.916667,176.0,25600.0
4,19521,J. Ibehre,35,England,61,61,Cambridge United,120.0,2,Right,1.0,2.0,ST,2017,6.333333,203.0,210.0


In [75]:
# для соблюдения синтаксиса переименнуем колонку "Release Clause"
df.rename(columns = {'Release Clause':'release_c'}, inplace = True )
df.to_csv('upd_dataset_new.csv', index = False)

In [76]:
#автоматический подбор типа данных
sample = pd.read_csv('/content/upd_dataset_new.csv', usecols=['Value','release_c'])
sample.convert_dtypes().dtypes
df.to_csv('upd_dataset_new.csv', index = False)


Большое количество пропусков зафиксировано в колонке Value. Также выявлена высокая корелляция между Value и Release Clause. Попробуем заполнить Value, используя медианное значение Release Clause.

In [77]:
med = df.release_c.median()
df['release_c'] = df['release_c'].fillna(med)


In [78]:
value_list = list(df.Value.value_counts().index)
for val in value_list:
  res_median = df[df['Value'] ==val]['release_c'].median()
  df[df['Value'] ==val]['release_c'].fillna(res_median, inplace = True)
df['Value'].isnull().sum()


257

In [79]:
df.to_csv('upd_dataset_new.csv', index = False)
df.Value

0        4000.0
1        1500.0
2         750.0
3       15500.0
4         120.0
         ...   
2394      625.0
2395      450.0
2396      975.0
2397      825.0
2398      875.0
Name: Value, Length: 2399, dtype: float64

### 2. Найдите в датафрейме полные дубликаты и удалите их. Значения могут быть одинаковыми, но написаны по-разному. Например, может отличаться размер регистра (заглавные и строчные буквы). Особое внимание уделить колонке с названиями команд.

In [86]:
df = df.applymap(lambda x: x.lower() if type(x) == str else x)
# приведем все записи к одному регистру
df.head(5)

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Skill Moves,Position,Joined,Height,Weight,release_c,Age_Group
0,1179,g. buffon,40,italy,88,88,paris saint-germain,4000.0,77,right,4.0,1.0,gk,2018,6.333333,203.0,7400.0,более 36
1,5479,casillas,37,spain,82,82,fc porto,1500.0,10,left,4.0,1.0,NaN,2015,6.083333,185.0,3000.0,более 36
2,8205,braga,35,portugal,70,70,cd aves,750.0,5,right,1.0,3.0,lcm,2017,5.916667,165.0,1600.0,30-36
3,9014,a. robben,34,netherlands,84,84,fc bayern münchen,15500.0,110,left,4.0,4.0,rm,2009,5.916667,176.0,25600.0,30-36
4,19521,j. ibehre,35,england,61,61,cambridge united,120.0,2,right,1.0,2.0,st,2017,6.333333,203.0,210.0,30-36


In [81]:
duplicates = df[df.duplicated()] #поиск дубликатов
print(duplicates)

         ID        Name  Age Nationality  Overall  Potential          Club  \
154  168651  i. rakitić   30     croatia       87         87  fc barcelona   
304  181872    a. vidal   31       chile       85         85  fc barcelona   

       Value  Wage Preferred Foot  International Reputation  Skill Moves  \
154  46500.0   260          right                       4.0          3.0   
304  26000.0   205          right                       4.0          3.0   

    Position  Joined    Height  Weight  release_c  
154      rcm    2014  6.000000   172.0    95300.0  
304      cam    2018  5.916667   165.0    53300.0  


In [82]:
df_no_dublicates = df.drop_duplicates() #удаление дубликатов

df_no_dublicates.to_csv('/content/upd_dataset_new.csv', index = False)
df_no_dublicates.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2394    False
2395    False
2396    False
2397    False
2398    False
Length: 2397, dtype: bool

In [54]:
df.describe()

,ID,Age,Overall,Potential,Value,Wage,International Reputation,Skill Moves,Joined,Height,Weight,release_c
count,2399.000000,2399.000000,2399.000000,2399.000000,2142.000000,2399.000000,2389.000000,2393.000000,2399.000000,2399.000000,2399.000000,2399.000000
mean,212033.680700,25.213005,69.115048,74.115048,5980.873016,24.178825,1.308497,2.469703,2016.145894,5.955300,166.802782,10681.417366
std,30549.399068,4.633622,8.060195,7.190957,11963.420681,49.552098,0.687175,0.844531,2.085829,0.219249,15.241015,21767.763702
min,1179.000000,16.000000,47.000000,48.000000,20.000000,0.000000,1.000000,1.000000,2000.000000,5.250000,121.000000,35.000000
25%,195105.000000,22.000000,64.000000,69.000000,500.000000,1.000000,1.000000,2.000000,2016.000000,5.833333,154.000000,1100.000000
50%,218745.000000,25.000000,69.000000,74.000000,1100.000000,5.000000,1.000000,2.000000,2017.000000,6.000000,165.000000,3800.000000
75%,234576.000000,29.000000,74.000000,79.000000,6000.000000,22.000000,1.000000,3.000000,2018.000000,6.083333,176.000000,7500.000000
max,246608.000000,44.000000,94.000000,95.000000,118500.000000,565.000000,5.000000,5.000000,2018.000000,6.583333,214.000000,228100.000000


In [83]:
# Минимальный возраст футболистов 16 лет, максимальный 44 года. Постороим функцию для разбивки возраста на заданные категории
def age_buckets(age):
  if age < 20:
    return 'до 20'
  elif age < 31:
    return '20-30'
  elif age < 37:
    return '30-36'
  elif age < 45:
    return 'более 36'
  else: return 'other'



In [87]:
df['Age_Group'] = df['Age'].apply(age_buckets) # добавим новый столбец
df['Age_Group'].value_counts() # посмотрим какое кол-во футболистов попало в каждую категорию


20-30       1774
30-36        340
до 20        270
более 36      15
Name: Age_Group, dtype: int64

In [85]:
df.to_csv('/content/upd_dataset_new.csv', index = False)